Crop REMA stripes by the polygon

In [47]:
import numpy as np
import rasterio as rio
import rasterio.mask
import tarfile
import fiona

import os
import sys
import geopandas as gpd
import shapely as sp
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt


#working_dir = '/Volumes/arc_02/REMOTE_SENSING/'
working_dir = "/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/test_stripe/"

sys.path.append(os.path.abspath(working_dir))
sys.path.append(os.path.abspath('/Users/home/whitefar/DATA/code/'))
sys.path.append(os.path.abspath('/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/'))
os.chdir(working_dir)

In [48]:
zipped_stripes_path = "/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/test_stripe/"
cropped_stripes_path = "/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/cropped_stripes/"

zipped_stripes_fnames = [f for f in os.listdir(zipped_stripes_path) if os.path.isfile(os.path.join(zipped_stripes_path, f))]

stripes_names = [stripe[:-7] for stripe in zipped_stripes_fnames]

tiffs_path = "/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/stripe_tiffs/" #save them temporararily

stripes_names

['SETSM_WV01_20151102_10200100437EF000_10200100477F7400_seg1_2m_v1.0']

In [49]:
with fiona.open("/Users/home/whitefar/DATA/Channel/line_channel_traced_for_50kms/channel_area.shp", "r") as shapefile:
    channel_area = [feature["geometry"] for feature in shapefile]

In [ ]:
def crop_stripes(stripe_name):

In [50]:
for i, stripe_name in enumerate(stripes_names):
    
    zipped_stripe_path =  zipped_stripes_path + zipped_stripes_fnames[i] 
    
    tiff_stripe_fname = stripe_name + "_dem.tif"
    
    with tarfile.open(zipped_stripe_path) as tar:
        stripe_tiff = tar.extract(member=tiff_stripe_fname, path=tiffs_path)
        
    with rio.open(tiffs_path + tiff_stripe_fname) as src:
        out_image, out_transform = rasterio.mask.mask(src, channel_area,crop=True)
        out_meta = src.meta.copy()
    
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
    cropped_stripe_path = cropped_REMA_stripes_path + "cropped" + tiff_stripe_fname
    
    with rio.open(cropped_stripe_path, "w", **out_meta) as dest:
        dest.write(out_image)
        
    os.remove(tiffs_path + tiff_stripe_fname)
        
    print(stripe_name + "cropped and written to "+ cropped_stripe_path)

SETSM_WV01_20151102_10200100437EF000_10200100477F7400_seg1_2m_v1.0_dem.tif
